# Bedrock AgentCore Memory 検証: 長期記憶の成功実証

このNotebookは、Bedrock AgentCore Memoryの「Episode（エピソード）」機能を検証するスクリプトです。
過去のトラブルシューティング（環境変数の設定漏れ）を記憶させ、現在の問い合わせに対して**「過去の失敗」を踏まえたアドバイスができるか**を確認します。

In [ ]:
# 1. 必要なライブラリのインポート
import uuid
import time
import json
import boto3
from datetime import datetime

try:
    from bedrock_agentcore.memory import MemoryClient
    print("Successfully imported MemoryClient.")
except ImportError:
    print("Installing bedrock-agentcore...")
    !pip install bedrock-agentcore strands-agents strands-agents-tools --quiet
    from bedrock_agentcore.memory import MemoryClient

# クライアント初期化
REGION = "us-east-1" 
memory_client = MemoryClient(region_name=REGION)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=REGION)

# 識別用ID
ACTOR_ID = "engineer_tanaka_001"

## 2. メモリリソース作成関数
比較のため、「Episode機能なし」と「Episode機能あり」の2つのメモリを作成します。

In [ ]:
def create_memory_resource(enable_episode=False):
    unique_id = str(uuid.uuid4())[:6]
    memory_name = f"trouble_mem_{unique_id}"
    
    strategies = []
    
    # Episode機能のON/OFF切り替え
    if enable_episode:
        strategies.append({
            "episodicMemoryStrategy": {
                "name": "episodeStrategy",
                # エピソード（セッションごとの要約）
                "namespaces": ["/strategies/{memoryStrategyId}/actors/{actorId}/sessions/{sessionId}"],
                # Reflection（洞察）: エピソードの親階層に設定
                "reflectionConfiguration": {
                     "namespaces": ["/strategies/{memoryStrategyId}/actors/{actorId}"]
                }
            }
        })
        label = "[Episode ON]"
    else:
        # 比較用: 戦略が空（記憶機能なし、あるいはセマンティックのみ）
        # ここでは純粋な比較のため空リスト、またはSemanticのみ定義してもOK
        label = "[Episode OFF]"

    print(f"Creating Memory {label}: {memory_name} ...")
    memory_info = memory_client.create_memory_and_wait(
        name=memory_name,
        description=f"Troubleshooting memory {label}",
        strategies=strategies
    )
    return memory_info['id']

# 2つのメモリを作成
mem_id_off = create_memory_resource(enable_episode=False)
mem_id_on = create_memory_resource(enable_episode=True)

## 3. 過去の日報データの投入
「環境変数忘れ」の失敗談を注入します。

In [ ]:
def inject_trouble_reports(memory_id):
    # --- Episode 1: 過去の失敗 ---
    session_id_1 = f"daily_report_20241001_{str(uuid.uuid4())[:4]}"
    messages_1 = [
        ("今日の日報です。AWS Lambdaの実装で3時間もハマってしまいました。", "USER"),
        ("それは大変でしたね。どのようなエラーでしたか？", "ASSISTANT"),
        ("実行時にキーエラーが出て動かなかったんです。コードは合ってるはずなのに。", "USER"),
        ("原因は特定できましたか？", "ASSISTANT"),
        ("はい。結局、Lambdaのコンソール画面で環境変数 `DB_CONNECTION_STRING` を設定し忘れてたのが原因でした。", "USER"),
        ("なるほど、環境変数の設定漏れはよくあるミスですね。記録しておきます。", "ASSISTANT")
    ]
    
    print(f"Injecting Episode 1 (Env Var Failure) -> {memory_id} ...")
    memory_client.create_event(
        memory_id=memory_id,
        actor_id=ACTOR_ID,
        session_id=session_id_1,
        messages=messages_1
    )
    print("Done.")

print("--- Processing Episode OFF Memory ---")
inject_trouble_reports(mem_id_off)

print("\n--- Processing Episode ON Memory ---")
inject_trouble_reports(mem_id_on)

## 4. 記憶の取得とエージェント回答 (修正済み)

ここが重要ポイントです。
AgentCore Memoryがデータを処理するのを待機した後、**`retrieve_memories`** を使って記憶を検索します。

In [ ]:
print("Waiting for AgentCore processing (sleeping 60s)...")
time.sleep(60)

def get_memory_context(memory_id):
    """
    メモリ内の情報を取得してテキスト化するヘルパー関数
    """
    context_text = ""
    
    try:
        # 1. メモリ設定の取得
        mem_details = memory_client.get_memory(memoryId=memory_id)
        memory_data = mem_details.get('memory', {})
        strategies = memory_data.get('strategies', [])
        
        target_strategy_id = None
        
        # Strategyの特定
        for s in strategies:
            if s.get('name') == 'episodeStrategy' or 'episodicMemoryStrategy' in s:
                target_strategy_id = s.get('strategyId')
                break 

        # --- A. Episode Strategyがある場合 ---
        if target_strategy_id:
            # 検索対象のNamespace: 全セッションを対象にするためsessionsまで指定
            namespace = f"/strategies/{target_strategy_id}/actors/{ACTOR_ID}/sessions"
            
            print(f"DEBUG: Querying memories via SDK from: {namespace}")

            try:
                # 今回のトラブルに関連しそうなキーワードで検索
                results = memory_client.retrieve_memories(
                    memory_id=memory_id,
                    namespace=namespace,
                    query="error failure Lambda trouble",
                    top_k=5
                )
                
                if results:
                    print(f"  [Episode ON] {len(results)}個のエピソードが見つかりました")
                    for item in results:
                        # 構造: item['content']['text'] に要約が入ります
                        content = item.get('content', {})
                        text = content.get('text')
                        timestamp = item.get('memoryTime', '')
                        
                        if text:
                            context_text += f"- 過去の記憶({timestamp}): {text}\n"
                else:
                    print("  [Episode ON] エピソードは見つかりませんでした (ヒットなし)")
                    
            except Exception as e:
                print(f"  [Warning] retrieve_memories failed: {e}")
                
        else:
            print("  [Warning] Episode Strategyの設定が見つかりませんでした")

    except Exception as e:
        print(f"  Error fetching memory: {e}")
        
    if not context_text:
        context_text = "(特に関連する記憶はありません)"
        
    return context_text

def ask_agent(memory_id, user_query, title):
    print(f"\n{'='*60}")
    print(f" {title}")
    print(f"{'='*60}")

    # 1. 記憶を取得
    context = get_memory_context(memory_id)
    
    # 2. プロンプト作成
    prompt = f"""
あなたは優秀なシニアエンジニアのアシスタントです。
以下の【過去の記憶】を参考にして、ユーザーの困りごとに対してアドバイスをしてください。
もし記憶の中に、今回のトラブルの原因になりそうな「過去の失敗パターン」があれば、それを具体的に指摘してください。

【過去の記憶】
{context}

ユーザー: {user_query}
AI:"""

    # 3. Amazon Nova Micro で回答生成
    try:
        model_id = "amazon.nova-micro-v1:0"
        body = json.dumps({
            "inferenceConfig": {"max_new_tokens": 512},
            "messages": [
                {"role": "user", "content": [{"text": prompt}]}
            ]
        })

        response = bedrock_runtime.invoke_model(modelId=model_id, body=body)
        response_body = json.loads(response.get('body').read())
        ai_response = response_body['output']['message']['content'][0]['text']
        
        print(f"\n  [🤖 AIエージェントの回答 (Nova Micro)]")
        print(f"  {ai_response}")
        
    except Exception as e:
        print(f"  Invoke Error: {e}")


# 検証実行！
query = "Lambdaが動かないんだよね〜なんでだろう"

ask_agent(mem_id_off, query, "❌ Episode OFF の場合")
ask_agent(mem_id_on, query, "✅ Episode ON の場合")

In [ ]:
# クリーンアップ
try:
    memory_client.delete_memory(memory_id=mem_id_off)
    memory_client.delete_memory(memory_id=mem_id_on)
    print("Cleanup complete.")
except:
    pass